In [0]:

H = 9
'''
Change H value to select different hyperparameters
 H = 1  lr = 0.001, # of epoch = 30 and step size = 20 
 H = 2  lr = 0.001, # of epoch = 20 
 H = 3  lr = 0.005, # of epoch = 20 
 H = 4  lr = 0.01, # of epoch = 20 
 H = 5  lr = 0.01, # of epoch = 100
 H = 6  lr = 0.01  # of epoch = 60 and stepsize = 40
 H = 7  lr = 0.01  # of epoch = 10 
 H = 8  lr = 0.01  # of epoch = 10 and step size = 5
 H = 9  lr = 0.001  # of epoch = 20 and step size = 10  batch size = 16
 H = 10  lr = 0.001  # of epoch = 10 and step size = 5  batch size = 64
'''


PRETRAINED = True
'''
  PRETRAINED = True   use the network pretrained on imagenet
  PRETRAINED = False   Training from scratch
'''

FREEZE = None
'''
  select part of the network to freeze
  FREEZE = None    training all the layers
  FREEZE = "FClayers"    training only the convolutional layers
  FREEZE = "CNNlayers"     training only the full connected layers

'''

DATA_IN_MEMORY = True
'''
  set DATA_IN_MEMORY = True  to load images on RAM 
  
'''


TRANSFORMATIONS = 1
'''
Modify tranformations to perform data argumentation
  TRANSFORMATIONS = 1 no data argumentation
  TRANSFORMATIONS = 2 use transforms.RandomHorizontalFlip
  TRANSFORMATIONS = 3 use transforms.RandomHorizontalFlip, transforms.RandomVerticalFlip, transforms.ColorJitte
  TRANSFORMATIONS = 4 use transforms.RandomRotation, transforms.RandomHorizontalFlip, transforms.ColorJitter
  TRANSFORMATIONS = 5 use transforms.RandomPerspective,transforms.RandomVerticalFlip
'''


MODEL = 'ResNet'
'''
  MODEL = 'ResNet'   ResNet152
  MODEL = 'VGG'      VGG16
  MODEL = 'AlexNet'  AlexNet
'''

"\n  MODEL = 'ResNet'   ResNet152\n  MODEL = 'VGG'      VGG16\n  MODEL = 'AlexNet'  AlexNet\n"


**Install requirements**

In [0]:
!pip3 install --upgrade 'pillow'
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'



Requirement already up-to-date: pillow in /usr/local/lib/python3.6/dist-packages (6.2.1)


In [0]:

from torchvision.datasets import VisionDataset

from PIL import Image

import os
import os.path
# import sys


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


class Caltech(VisionDataset):
    
    def __init__(self, root, split='train', transform=None, target_transform=None):
        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)

        self.split = split # This defines the split you are going to use
                           # (split files are called 'train.txt' and 'test.txt')

        '''
        - Here you should implement the logic for reading the splits files and accessing elements
        - If the RAM size allows it, it is faster to store all data in memory
        - PyTorch Dataset classes use indexes to read elements
        - You should provide a way for the __getitem__ method to access the image-label pair
          through the index
        - Labels should start from 0, so for Caltech you will have lables 0...100 (excluding the background class) 
        '''
        # if not os.path.isdir('./Homework2-Caltech101'):
        #   !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git

        
        self.datadir = root
       
        self.filedir = 'Homework2-Caltech101/'+ split +'.txt'
        print(split)
        self.imageroots = []
        self.labels = []
        n = -1
        category = None
        f = open(self.filedir, "r")
        for x in f:
          newcategory = x.split('/')[0]
          if newcategory != 'BACKGROUND_Google':
            if newcategory != category:
              category = newcategory
              n = n+1
            if DATA_IN_MEMORY == True:
              self.imageroots.append(pil_loader(self.datadir + '/'+ x.rstrip()))
            
           

            else:
              self.imageroots.append(x.rstrip())
            self.labels.append(n)
            

    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''

        # image, label = ... # Provide a way to access image and label via index
        #                    # Image should be a PIL Image
        #                    # label can be int

        # Applies preprocessing when accessing the image
        import scipy.io
        label = self.labels[index]
        if DATA_IN_MEMORY == True:
          img = self.imageroots[index]
        else:
          img = pil_loader(self.datadir + '/'+ self.imageroots[index])
        
        img = self.transform(img)
          
       
      
        return img, label



    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        length = len(self.labels)
        return length

**Import libraries**

In [0]:
import os
import logging
# import dataclass



import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet
from torchvision.models import vgg
from PIL import Image
from tqdm import tqdm



In [0]:
def testModel(model,testset): 
  # net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  model.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in testset:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = model(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(testset.dataset))
  print(running_corrects)
  print('Test Accuracy: {}'.format(accuracy))
  model.train(True)
  return float(accuracy)

**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
NUM_CLASSES = 102 # 101 + 1: There is am extra Background class that should be removed 
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
LOG_FREQUENCY = 10 
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

if H == 1:
 
  BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 1e-3           # The initial Learning Rate
  NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 20      # How many epochs before decreasing learning rate (if using a step-down policy)
  
if H == 2:
 

  BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 1e-3            # The initial Learning Rate
  NUM_EPOCHS = 20      # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 20      # How many epochs before decreasing learning rate (if using a step-down policy)
  
  
if H == 3:
 
  BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 5e-3            # The initial Learning Rate
  NUM_EPOCHS = 20      # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 20      # How many epochs before decreasing learning rate (if using a step-down policy)
 
if H == 4:
 

  BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = e-2            # The initial Learning Rate
  NUM_EPOCHS = 20      # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 20     # How many epochs before decreasing learning rate (if using a step-down policy)
  

if H == 5:
 

  BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 1e-2            # The initial Learning Rate
  MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
  WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default
  NUM_EPOCHS = 100      # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 100      # How many epochs before decreasing learning rate (if using a step-down policy)
  
if H == 6:
  BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 1e-2            # The initial Learning Rate
  NUM_EPOCHS = 60      # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 40      # How many epochs before decreasing learning rate (if using a step-down policy)
 
if H == 7:
  BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 1e-2            # The initial Learning Rate
  NUM_EPOCHS = 10      # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 10      # How many epochs before decreasing learning rate (if using a step-down policy)
  
if H == 8:
  BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 1e-2            # The initial Learning Rate
  NUM_EPOCHS = 10     # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 5      # How many epochs before decreasing learning rate (if using a step-down policy)
  
if H == 9:
  BATCH_SIZE = 16    # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 1e-3            # The initial Learning Rate
  NUM_EPOCHS = 20     # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 20      # How many epochs before decreasing learning rate (if using a step-down policy)
  
if H == 10:
  BATCH_SIZE = 64   # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results
  LR = 1e-3            # The initial Learning Rate
 
  NUM_EPOCHS = 10     # Total number of training epochs (iterations over dataset)
  STEP_SIZE = 5      # How many epochs before decreasing learning rate (if using a step-down policy)
  

**Define Data Preprocessing**

In [0]:

if PRETRAINED == True:
  if TRANSFORMATIONS == 1:  
      train_transform= transforms.Compose([transforms.Resize((256)),
                                      transforms.CenterCrop((224,224)),            
                                      transforms.ToTensor(),                     
                                      transforms.Normalize(                      
                                      mean=[0.485, 0.456, 0.406],                
                                      std=[0.229, 0.224, 0.225]     )])
   
  if TRANSFORMATIONS == 2:
      train_transform= transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),   
                                      transforms.RandomHorizontalFlip(p=0.5),     
                                      transforms.ToTensor(),                     
                                      transforms.Normalize(                      
                                      mean=[0.485, 0.456, 0.406],                
                                      std=[0.229, 0.224, 0.225]     )])
  if TRANSFORMATIONS == 3:
       train_transform= transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),   
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      torchvision.transforms.RandomApply([transforms.ColorJitter(
            brightness=0.5,
            contrast=0.5,
            saturation=0.5)], p=0.5),
                                      transforms.ToTensor(),
                                      transforms.Normalize(                      
                                      mean=[0.485, 0.456, 0.406],                
                                      std=[0.229, 0.224, 0.225]     )                     
                                      ])

  if TRANSFORMATIONS == 4:
       train_transform= transforms.Compose([transforms.Resize(256),  
                                      transforms.CenterCrop(224),
                                      transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.4, contrast=0.4,saturation=0.4,),
                                      transforms.ToTensor(),                     
                                      transforms.Normalize(                      
                                      mean=[0.485, 0.456, 0.406],                
                                      std=[0.229, 0.224, 0.225]     )])     

  if TRANSFORMATIONS == 5:
       train_transform= transforms.Compose([transforms.Resize(256),  
                                      transforms.CenterCrop(224),
                                      transforms.RandomPerspective(distortion_scale = 0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.ToTensor(),                     
                                      transforms.Normalize(                      
                                      mean=[0.485, 0.456, 0.406],                
                                      std=[0.229, 0.224, 0.225]     )])   


  eval_transform= transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),                
                                      transforms.ToTensor(),                     
                                      transforms.Normalize(                      
                                      mean=[0.485, 0.456, 0.406],                
                                      std=[0.229, 0.224, 0.225]     )])
    
  

else:
  # Define transforms for training phase
  train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                        transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                    # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                    # Remember this when applying different transformations, otherwise you get an error
                                        transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
  ])
  # Define transforms for the evaluation phase
  eval_transform = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
  ])

**Prepare Dataset**

In [0]:
# Clone github repository with data
if not os.path.isdir('./Homework2-Caltech101'):
  !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git

DATA_DIR = 'Homework2-Caltech101/101_ObjectCategories'

# # Prepare Pytorch train/test Datasets
# train_dataset = torchvision.datasets.ImageFolder(DATA_DIR, transform=train_transform)
# test_dataset = torchvision.datasets.ImageFolder(DATA_DIR, transform=eval_transform)

# train_indexes = [idx for idx in range(len(train_dataset)) if idx % 5]
# test_indexes = [idx for idx in range(len(test_dataset)) if not idx % 5]

# train_dataset = Subset(train_dataset, train_indexes)
# test_dataset = Subset(test_dataset, test_indexes)

# # Check dataset sizes
# print('Train Dataset: {}'.format(len(train_dataset)))
# print('Test Dataset: {}'.format(len(test_dataset)))

train_dataset = Caltech(root = DATA_DIR, split='train', transform= train_transform)
test_dataset = Caltech(root = DATA_DIR, split='test', transform= eval_transform)
print('Train Dataset: {}'.format(len(train_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))
import numpy as np
from sklearn.model_selection import train_test_split

indices = np.arange(len(train_dataset))
x1, x2, y1, y2, idxtrain, idxeval = train_test_split(
    train_dataset.imageroots, train_dataset.labels, indices, test_size=1/2,stratify=train_dataset.labels, random_state = 42)
print(len(idxtrain))
print(idxtrain)


train
test
Train Dataset: 5784
Test Dataset: 2893
2892
[5256 3726 2273 ... 3143 4041 1686]


In [0]:
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

# import matplotlib as mpl
# mpl.rcParams['axes.grid'] = False
# mpl.rcParams['image.interpolation'] = 'nearest'
# mpl.rcParams['figure.figsize'] = 10, 15

# def show_dataset(dataset, n=5):
#   img = np.vstack((np.hstack((np.asarray(dataset[i][0]) for _ in range(n)))
#                    for i in range(len(dataset))))
#   plt.imshow(img.transpose(1, 2, 0))
#   plt.axis('off')

In [0]:
# try_dataset = Subset(train_dataset, [150])

# show_dataset(try_dataset)

In [0]:
eval_dataset = Subset(train_dataset, idxeval)
train_dataset = Subset(train_dataset, idxtrain)




In [0]:
print(len(idxeval))
print(eval_dataset)
print(train_dataset)
print(idxtrain)
print(idxeval)
print(eval_dataset.dataset)
print(train_dataset.dataset)

2892
[5256 3726 2273 ... 3143 4041 1686]
[1646 2213 3103 ... 4547 4330 5698]
Dataset Caltech
    Number of datapoints: 5784
    Root location: Homework2-Caltech101/101_ObjectCategories
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset Caltech
    Number of datapoints: 5784
    Root location: Homework2-Caltech101/101_ObjectCategories
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Prepare Network**

In [0]:

if PRETRAINED == True:
  if MODEL == 'AlexNet':
    net = torchvision.models.alexnet(pretrained=True)
  if MODEL == 'VGG':
    net = torchvision.models.vgg16(pretrained=True)
  if MODEL == 'ResNet':
   net = torchvision.models.resnet152(pretrained=True)


else:
  net = alexnet() # Loading AlexNet model

  # AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
  # We need 101 outputs for Caltech-101
if MODEL == 'AlexNet' or MODEL == 'VGG':
  net.classifier[6] = nn.Linear(4096, NUM_CLASSES-1) # nn.Linear in pytorch is a fully connected layer

                                                # The convolutional layer is nn.Conv2
else:
  net.fc = nn.Linear(2048,NUM_CLASSES-1)                                                 

  # We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
  # It is mandatory to study torchvision.models.alexnet source code


**Prepare Training**

In [0]:
  
  # Choose parameters to optimize
  # To access a different set of parameters, you have to access submodules of AlexNet
  # (nn.Module objects, like AlexNet, implement the Composite Pattern)
  # e.g.: parameters of the fully connected layers: net.classifier.parameters()
  # e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 

if FREEZE == 'FClayers':
  print(' training only the convolutional layers ')
  for param in net.classifier.parameters():
    param.requires_grad = False
  parameters_to_optimize = net.features[:].parameters()  # In this case we optimize only the full connected layers of AlexNet


if FREEZE == 'CNNlayers':
  print(' training only the full connected layers ')
  for param in net.features.parameters():
   param.requires_grad = False  
  parameters_to_optimize = net.classifier[:].parameters()  # In this case we optimize only the full connected layers of AlexNet

if FREEZE == None:
  print(' training all the layers ')
  parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

  # Define optimizer
  # An optimizer updates the weights based on loss
  # We use SGD with momentum
  # Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

  # Define scheduler
  # A scheduler dynamically changes learning rate
  # The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)



 training all the layers 


**Train**

In [0]:

# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
old_accuracy = 0
cudnn.benchmark # Calling this optimizes runtime
bestmodel = None
current_step = 0
acc = []
L = []
e = []
step = 0
totloss = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  e.append(epoch +1 )
  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)
    totloss = totloss + loss
    step = step + 1

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
       print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
  L.append(loss/step)  
  accuracy = testModel(net,eval_dataloader)
  acc.append(testModel(net,eval_dataloader))
  step = 0
  totloss = 0
  print('old', old_accuracy)
  if(accuracy > old_accuracy):
    epochs = epoch
    torch.save(net.state_dict(), 'model.pt')
    old_accuracy = accuracy
 
  # Step the scheduler
  scheduler.step() 



Starting epoch 1/20, LR = [0.001]
Step 0, Loss 4.580991268157959
Step 10, Loss 4.350844860076904
Step 20, Loss 3.7105305194854736
Step 30, Loss 3.658505916595459
Step 40, Loss 3.6082301139831543
Step 50, Loss 3.567479372024536
Step 60, Loss 2.845735549926758
Step 70, Loss 2.955547332763672
Step 80, Loss 2.3314735889434814
Step 90, Loss 3.33160400390625
Step 100, Loss 2.165052890777588
Step 110, Loss 2.3150315284729004
Step 120, Loss 1.9161498546600342
Step 130, Loss 1.86597740650177
Step 140, Loss 1.7593331336975098
Step 150, Loss 1.8062716722488403
Step 160, Loss 2.1780643463134766
Step 170, Loss 1.1705706119537354
2099
Test Accuracy: 0.7257952973720608
2099
Test Accuracy: 0.7257952973720608
old 0
Starting epoch 2/20, LR = [0.001]
Step 180, Loss 1.3859529495239258
Step 190, Loss 1.2771459817886353
Step 200, Loss 1.2261550426483154
Step 210, Loss 1.359933614730835
Step 220, Loss 1.4803320169448853
Step 230, Loss 1.258673906326294
Step 240, Loss 1.224975347518921
Step 250, Loss 0.964664

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax1 = plt.subplots()

ax1.set_xlabel('epoch')
ax1.set_ylabel('accuracy',color='orange')
ax1.plot(e, acc,color='orange')
ax1.tick_params(axis='y',labelcolor='orange')
ax1.set_xticks(e) 

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax2.set_ylabel('loss',color='blue')  # we already handled the x-label with ax1
ax2.plot(e,L,color='blue')
ax2.tick_params(axis='y',labelcolor='blue')
#ax2.tick_params(axis='x')

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.show()

In [0]:

net.load_state_dict(torch.load('model.pt'))
testModel(net,test_dataloader)
